In [2]:
# Training script for LazyConfig models
try:
    # ignore ShapelyDeprecationWarning from fvcore
    import warnings

    from shapely.errors import ShapelyDeprecationWarning

    warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

except:
    pass
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger

setup_logger()

import gc
import os
import time

import detectron2.utils.comm as comm

# import some common libraries
import numpy as np
import torch

# import some common detectron2 utilities
from detectron2.config import LazyConfig, get_cfg
import detectron2.data as data
from detectron2.engine import (
    launch,
)

from deepdisc.data_format.augment_image import train_augs
from deepdisc.data_format.image_readers import DC2ImageReader
from deepdisc.data_format.register_data import register_data_set#, register_loaded_data_set
from deepdisc.model.loaders import (
    RedshiftFlatDictMapper,
    return_test_loader,
    return_train_loader,
)
from deepdisc.model.models import RedshiftPointCasROIHeads, RedshiftPointROIHeads, RedshiftPDFROIHeads, return_lazy_model
from deepdisc.training.trainers import (
    return_evallosshook,
    return_lazy_trainer,
    return_optimizer,
    return_savehook,
    return_schedulerhook,
)
from deepdisc.utils.parse_arguments import make_training_arg_parser



In [3]:
#import importlib
#importlib.reload(rail.estimation.algos.deepdisc)

In [4]:
#from rail.estimation.algos.deepdisc import DeepDiscInformer
import rail
from rail.estimation.algos.deepdisc import *
from rail.core.data import TableHandle
from rail.core.stage import RailStage

from rail.deepdisc.configs import *

In [5]:
cfgfile = "/home/shared/hsc/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.py"
dirpath = './tests/deepdisc/test_data/dc2/'
output_dir = './'
output_name = 'test'

trainfile = dirpath + "flattened_data_test.npy"
testfile = dirpath + "flattened_data_test.npy"
classes = ['object']
numclasses = 1

In [6]:
cfg = get_lazy_config(cfgfile, 1, 1)
cfg.train.init_checkpoint

'detectron2://ImageNetPretrained/MSRA/R-50.pkl'

In [7]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [8]:
testdata = np.load('./tests/deepdisc/test_data/dc2/flattened_data_test.npy')
mapper = RedshiftFlatDictMapper().map_data

In [9]:
dataset_dicts={}
dds = []
for row in testdata:
    dds.append(mapper(row))
dataset_dicts['test'] = dds

In [10]:
training = DS.add_data("training", testdata, TableHandle)
testing = DS.add_data("testing", testdata, TableHandle)

In [11]:
deep_dict = dict(epochs=200,
                numclasses=1,
                batch_size=1,
                output_dir ='./',
                cfgfile="/home/shared/hsc/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.py",
                output_name ='test_informer')

In [12]:
#Inform = DeepDiscInformer.make_stage(name='Inform_DeepDISC', model='detectron2://ImageNetPretrained/MSRA/R-50.pkl', **deep_dict)
Inform = DeepDiscInformer.make_stage(name='Inform_DeepDISC', model='test_informer.pkl', **deep_dict)


In [13]:
Inform.inform(training)

RuntimeError: No CUDA GPUs are available

### Inference

In [13]:
#Estimator = DeepDiscEstimator.make_stage(name='DeepDiscEstimator',
#                                       model=Inform.get_handle('model'), **deep_dict)

Estimator = DeepDiscPDFEstimator.make_stage(name='DeepDiscEstimator',
                                       model=Inform.get_handle('model'), hdf5_groupname=None, **deep_dict)

In [14]:

results = Estimator.estimate(testing)



[11/16 13:27:33 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./test_informer.pth ...
Processing Data
Matching objects
Inserting handle into data store.  output_DeepDiscEstimator: inprogress_output_DeepDiscEstimator.hdf5, DeepDiscEstimator


In [15]:
res = results.read()

In [16]:
res